In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pickle
import re

#from the folders results_real load all csv files that start with dataFrame_results and put them in a dictionary with the name of the file as key
# Path: results_processing.ipynb
def load_results(path):
    results = {}
    for file in os.listdir(path):
        if file.startswith('dataFrame_results'):
            results[file[len("dataFrame_results"):-len('.csv')]] = pd.read_csv(os.path.join(path, file))
    return results

#load the results from the folder results_real
# Path: results_processing.ipynb
results = load_results('results_synthetic_10days')



In [11]:
mean_std_dictionary = {}
for metric, dataset in results.items():
    #put dataset.mean and dataset.std in a dataframe with the column names mean and std
    df = pd.DataFrame({'mean': dataset.mean(), 'std': dataset.std()})
    df = df.round(2)
    #put the dataframe in a dictionary with the name of the metric as key
    mean_std_dictionary[metric] = df

/tmp/ipykernel_70776/1114752087.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = pd.DataFrame({'mean': dataset.mean(), 'std': dataset.std()})
/tmp/ipykernel_70776/1114752087.py:4: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = pd.DataFrame({'mean': dataset.mean(), 'std': dataset.std()})
/tmp/ipykernel_70776/1114752087.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only

In [13]:
# Combine all the dataframes into one dataframe
df = pd.concat(mean_std_dictionary, axis=0)

# Unstack the dataframe to have models as columns and (metric, mean) and (metric, std) as multi-level indexes
unstacked_df = df.unstack()

# Transpose the dataframe to have models as columns and metrics as rows
unique_df = unstacked_df
unique_df = unique_df.swaplevel(0, 1, axis=1).sort_index(axis=0)

#reored the column putting mean and std besides
unique_df = unique_df.reindex(['mean', 'std'], axis=1, level=1)

unique_df = unique_df.reorder_levels([0, 1], axis=1)
unique_df = unique_df.sort_index(axis=1)

#save to csv
unique_df.to_csv('synthetic_results_10d.csv', index=True, header=True)


# Regressor scores

In [14]:
def load_results(path):
    results = {}
    for file in os.listdir(path):
        if file.startswith('regressor_scores_'):
            results[file[len("regressor_scores_"):-len('.csv')]] = pd.read_csv(os.path.join(path, file))
    return results

#load the results from the folder results_real
# Path: results_processing.ipynb
results = load_results('results_real10days')

In [15]:
df = pd.concat(results.values(), keys=results.keys())

In [16]:
grouped = df.groupby(level=1)

result = grouped.agg(
    X_TERMINAL_mean=('X_TERMINAL', np.mean),
    X_TERMINAL_std=('X_TERMINAL', np.std),
    Y_TERMINAL_mean=('Y_TERMINAL', np.mean),
    Y_TERMINAL_std=('Y_TERMINAL', np.std),
    TX_AMOUNT_mean=('TX_AMOUNT', np.mean),
    TX_AMOUNT_std=('TX_AMOUNT', np.std),
).reset_index(drop=True)


In [17]:
result.round(2).to_csv('synthetic_results_10d_regressors.csv', index=True, header=True)